In [2]:
import numpy as np
import torch
import torch.nn as nn

from torch.autograd import Variable

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI
from pyro.optim import Adam

# Linear dataset generation

In [3]:
N = 100  # size of toy data
p = 1    # number of features

def build_linear_dataset(N, noise_std=0.1):
    X = np.linspace(-6, 6, num=N)
    y = 3 * X + 1 + np.random.normal(0, noise_std, size=N)
    X, y = X.reshape((N, 1)), y.reshape((N, 1))
    X, y = Variable(torch.Tensor(X)), Variable(torch.Tensor(y))
    return torch.cat((X, y), 1)

# Define Linear Model

In [4]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)

    def forward(self, x):
        return self.linear(x)

regression_model = RegressionModel(p)

# Train Linear Model

In [5]:
loss_fn = torch.nn.MSELoss(size_average=False)
optim = torch.optim.Adam(regression_model.parameters(), lr=0.01)
num_iterations = 500

def main():
    data = build_linear_dataset(N, p)
    print(data.shape)
    x_data = data[:, :-1]
    y_data = data[:, -1]
    for j in range(num_iterations):
        # run the model forward on the data
        y_pred = regression_model(x_data)
        # calculate the mse loss
        loss = loss_fn(y_pred, y_data)
        # initialize gradients to zero
        optim.zero_grad()
        # backpropagate
        loss.backward()
        # take a gradient step
        optim.step()
        if (j + 1) % 50 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss.data[0]))
    # Inspect learned parameters
    print("Learned parameters:")
    for name, param in regression_model.named_parameters():
        print("%s: %.3f" % (name, param.data.numpy()))

if __name__ == '__main__':
    main()

torch.Size([100, 2])
[iteration 0050] loss: 4357.6592
[iteration 0100] loss: 2609.5933
[iteration 0150] loss: 1484.0399
[iteration 0200] loss: 809.0546
[iteration 0250] loss: 435.0489
[iteration 0300] loss: 244.8268
[iteration 0350] loss: 156.4057
[iteration 0400] loss: 118.9292
[iteration 0450] loss: 104.4559
[iteration 0500] loss: 99.3632
Learned parameters:
linear.weight: 2.915
linear.bias: 0.905


# Bayesian Regression

## Model Lift

* Turn non-baysian model to baysian model by random_module()
* sample a model
* run point evaluation by a sample model

In [6]:

def model(data):
    
    # Create unit normal priors over the parameters
    mu, sigma = Variable(torch.zeros(1, p)), Variable(10 * torch.ones(1, p))
    bias_mu, bias_sigma = Variable(torch.zeros(1)), Variable(10 * torch.ones(1))
    w_prior, b_prior = Normal(mu, sigma), Normal(bias_mu, bias_sigma)
    priors = {'linear.weight': w_prior, 'linear.bias': b_prior}
    
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", regression_model, priors)
    
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    
    # run the regressor forward conditioned on data
    x_data = data[:, :-1]
    y_data = data[:, -1]
    prediction_mean = lifted_reg_model(x_data).squeeze()
    
    # condition on the observed data
    pyro.sample("obs",
                Normal(prediction_mean, Variable(0.1 * torch.ones(data.size(0)))),
                obs=y_data.squeeze())

# Guide 

* To SVI (stochastic variational inference), need guide, i.e, a parameterized family of distributions 

In [7]:
softplus = torch.nn.Softplus()

def guide(data):

    # define our variational parameters
    w_mu = Variable(torch.randn(1, p), requires_grad=True)
    # note that we initialize our sigmas to be pretty narrow
    w_log_sig = Variable(-3.0 * torch.ones(1, p) + 0.05 * torch.randn(1, p),
                         requires_grad=True)
    b_mu = Variable(torch.randn(1), requires_grad=True)
    b_log_sig = Variable(-3.0 * torch.ones(1) + 0.05 * torch.randn(1),
                         requires_grad=True)
    
    # register learnable params in the param store
    mw_param = pyro.param("guide_mean_weight", w_mu)
    sw_param = softplus(pyro.param("guide_log_sigma_weight", w_log_sig))
    mb_param = pyro.param("guide_mean_bias", b_mu)
    sb_param = softplus(pyro.param("guide_log_sigma_bias", b_log_sig))
    
    # guide distributions for w and b
    w_dist, b_dist = Normal(mw_param, sw_param), Normal(mb_param, sb_param)
    dists = {'linear.weight': w_dist, 'linear.bias': b_dist}
    
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    
    # sample a regressor (which also samples w and b)
    return lifted_module()

# Inference

In [8]:
optim = Adam({"lr": 0.01})
svi = SVI(model, guide, optim, loss="ELBO")

def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N, p)
    for j in range(num_iterations):
        # calculate the loss and take a gradient step
        loss = svi.step(data)
        if j % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss / float(N)))

if __name__ == '__main__':
    main()

[iteration 0001] loss: 2791.8513
[iteration 0101] loss: 761.1793
[iteration 0201] loss: 181.3087
[iteration 0301] loss: 58.4005
[iteration 0401] loss: 50.9612


# See Results of inference

In [9]:
for name in pyro.get_param_store().get_all_param_names():
    print("[%s]: %.3f" % (name, pyro.param(name).data.numpy()))

[guide_mean_weight]: 3.036
[guide_log_sigma_weight]: -3.147
[guide_mean_bias]: 1.046
[guide_log_sigma_bias]: -3.234


# Point Evaluation

In [10]:
X = np.linspace(6, 7, num=20)
y = 3 * X + 1
X, y = X.reshape((20, 1)), y.reshape((20, 1))
x_data, y_data = Variable(torch.Tensor(X)), Variable(torch.Tensor(y))
loss = nn.MSELoss()
y_preds = Variable(torch.zeros(20, 1))
for i in range(20):
    # guide does not require the data
    sampled_reg_model = guide(None)
    # run the regression model and add prediction to total
    y_preds = y_preds + sampled_reg_model(x_data)
# take the average of the predictions
y_preds = y_preds / 20
print ("Loss: ", loss(y_preds, y_data).data[0])

Loss:  0.060590557754039764
